# Обучение реранкера и сабмит

# Импорты и параметры

In [1]:
# !pip install -q git+https://github.com/sb-ai-lab/LightAutoML.git

In [2]:
# подключение к диску
import os
import sys
from google.colab import drive

if 'drive' not in os.listdir():
    drive.mount('/content/drive')

In [3]:
# основные пути до папок с данными и моделями + параметры процесса
data_path = '/content/drive/MyDrive/hse/hse_recsys_kaggle/data'
model_path = '/content/drive/MyDrive/hse/hse_recsys_kaggle/models'
# основные поля событий
candidates_files = ['sas4rec_models_candidates.csv']#, 'bert4rec_models_candidates.csv']
#  ['bert4rec_models_candidates.csv', 'slim_models_candidates.csv', 'iknn_models_candidates.csv',
#                     'als_models_candidates.csv', 'sas4rec_models_candidates.csv']
val_file = 'val.csv'
test_file = 'test.csv'
features_file = 'features_reranker.csv'
items_features_file = 'item_features.csv'
emb_file = 'sas4rec_user_embeddings.csv'
user_col = 'user_id'
item_col = 'item_id'
time_col = 'timestamp'
interaction_col = 'rating'
random_state = 6
K=10

In [4]:
# standard libraries
import sys
import os
import warnings
warnings.filterwarnings('ignore')
# ds libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
# lama
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
# save/load model
import joblib

# Загрузка данных:
- кандидатов моделей
- фичей и эмбеддингов юзеров
- вал и тест


In [5]:
# кандидаты
models_candidates = []
for cf in candidates_files:
    candidates = pd.read_csv(os.path.join(data_path, cf))
    candidates['model'] = cf.split('_')[0]
    candidates[interaction_col] = candidates[cf.split('_')[0]]
    models_candidates.append(candidates)

# все скоры соберем в разные столбцы для user_id, item_id
models_candidates = pd.concat(models_candidates).reset_index(drop=True) \
      .pivot(index=[user_col, item_col], columns='model', values=interaction_col, ).reset_index()
models_candidates.columns = models_candidates.columns.values

# val и test
val = pd.read_csv(os.path.join(data_path, val_file)).drop(columns=[time_col])
test = pd.read_csv(os.path.join(data_path, test_file)).drop(columns=[time_col])

# positives
positives = models_candidates.merge(test, how='inner', on=[user_col,item_col])
positives['target'] = 1
# negatives
negatives = models_candidates.merge(test, how = 'left', on=[user_col,item_col])
negatives = negatives.loc[negatives[interaction_col].isnull()].sample(frac = .5)
negatives['target'] = 0


reranker_df = pd.concat([positives, negatives]).reset_index(drop=True)
# загружаем признаки
items_features = pd.read_csv(os.path.join(data_path, items_features_file))
user_features = pd.read_csv(os.path.join(data_path, features_file)).drop_duplicates(subset='user_id').drop(columns=items_features)
emb = pd.read_csv(os.path.join(data_path, emb_file))

reranker_df = reranker_df.merge(user_features[[user_col,'age','gender']], how='left', on=[user_col]) \
                .merge(items_features, how='left', on=[item_col])


# Обучение реранкера

## Определение модели

In [6]:
config_model = dict(
                    n_threads = 1,
                    n_folds = 5,
                    random_state = 42,
                    task = dict(name='binary',metric='auc', loss='logloss',greater_is_better=True),
                    general_params = {'use_algos': [['cb','xgb','lgb']],
                                      'weighted_blender_max_nonzero_coef' : 0.05,},
                    advanced_roles = False,
                    roles = {'target': 'target',
                             'category': ['age','gender'],
                             'drop' : [user_col, item_col, time_col, interaction_col]},

                    model_name = f'reranker.pkl',
    )

In [7]:
task = Task(name=config_model['task']['name'],
                metric=config_model['task']['metric'],
                loss=config_model['task']['loss'],
                greater_is_better=config_model['task']['greater_is_better'])
reader_params = {'n_jobs' : config_model['n_threads'],
                 'cv': config_model['n_folds'],
                 'random_state' : config_model['random_state'],
                 'advanced_roles' : config_model['advanced_roles']
                 }

reranker = TabularAutoML(
        task = task,
        timeout= 60*60*10,
        cpu_limit=-1,
        general_params=config_model['general_params'],
        reader_params=reader_params,
        tuning_params=config_model.get('tuning_params', {'max_tuning_iter': 5}),
        selection_params={'mode': 1, 'importance_type': 'permutation','fit_on_holdout': True, 'cutoff': 0, 'select_algos': [ 'gbm','linear_l2','rf']},
        cb_params = {'default_params': {'num_trees': 100}},
        xgb_params = {'default_params': {'n_estimators': 100}},
        lgb_params = {'default_params': {'num_trees': 100}},
)

cv = StratifiedKFold(n_splits=config_model['n_folds'],shuffle=True, random_state=config_model['random_state'])

## Обучение модели

In [8]:
%%time
train_pred = reranker.fit_predict(reranker_df,
                               roles=config_model['roles'],
                               verbose=1,
                               cv_iter=list(cv.split(reranker_df,reranker_df[config_model['roles']['target']])))

[07:21:51] Stdout logging level is INFO.


INFO:lightautoml.automl.presets.base:Stdout logging level is INFO.


[07:21:51] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer


[07:21:51] Task: binary



INFO:lightautoml.automl.presets.base:Task: binary



...

[07:23:03] Automl preset training completed in 72.10 seconds



INFO:lightautoml.automl.presets.base:Automl preset training completed in 72.10 seconds



[07:23:03] Model description:
Final prediction for new objects (level 0) = 
	 0.32053 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LightGBM) +
	 0.56837 * (5 averaged models Lvl_0_Pipe_0_Mod_1_CatBoost) +
	 0.11111 * (5 averaged models Lvl_0_Pipe_0_Mod_2_XGBoost) 



INFO:lightautoml.automl.presets.base:Model description:
Final prediction for new objects (level 0) = 
	 0.32053 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LightGBM) +
	 0.56837 * (5 averaged models Lvl_0_Pipe_0_Mod_1_CatBoost) +
	 0.11111 * (5 averaged models Lvl_0_Pipe_0_Mod_2_XGBoost) 



CPU times: user 1min 53s, sys: 2.43 s, total: 1min 56s
Wall time: 1min 12s


In [9]:
target = reranker.reader.target
train_pred = reranker.predict(reranker_df)
print(f"Целевая метрика {config_model['task']['metric']} на train для {target}:",
      reranker.task.metric_func(reranker_df[target],train_pred.data.reshape(-1)))
print('Количество используемых фичей:', len(reranker.reader.used_features))
print('Фичи:', reranker.reader.used_features)

Целевая метрика auc на train для target: 0.752063745470581
Количество используемых фичей: 14
Фичи: ['genre_7', 'genre_17', 'genre_15', 'genre_13', 'genre_1', 'sas4rec', 'genre_2', 'genre_0', 'genre_11', 'gender', 'genre_3', 'genre_6', 'genre_4', 'genre_12']


## Оценка модели

In [10]:
# предсказание
reranker_df['reranker_score'] = reranker.predict(reranker_df).data.reshape(-1)
reranker_df = reranker_df.sort_values(by = [user_col, 'reranker_score'], ascending = [True, False])
reranker_df['rank'] = reranker_df.groupby(user_col).cumcount() + 1
reranker_df.head()

,user_id,item_id,sas4rec,rating,target,age,gender,genre_0,genre_1,genre_2,...,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17,reranker_score,rank
137654,0,2275,5.597428,NaN,0,35,M,1,1,0,...,0,0,0,0,0,1,0,0,0.018846,1
147414,0,1461,5.982721,NaN,0,35,M,0,1,0,...,0,0,0,0,0,0,0,0,0.018255,2
10862,0,1171,3.792193,NaN,0,35,M,1,0,0,...,0,0,0,0,0,1,0,0,0.015913,3
6988,0,1044,4.957140,NaN,0,35,M,1,1,0,...,0,0,0,0,1,1,0,0,0.014628,4
111027,0,1245,4.602338,NaN,0,35,M,1,1,0,...,0,0,0,0,0,0,0,0,0.014220,5


In [11]:
# метрики на тесте
# prepare dataset
df = test.set_index([user_col, item_col]).join(reranker_df[[user_col, item_col, 'rank']].set_index([user_col, item_col]))
df = df.sort_values(by = [user_col, 'rank'])
df['users_watch_count'] = df.groupby(level = user_col)['rank'].transform(np.size)
df['cumulative_rank'] = df.groupby(level = user_col).cumcount() + 1
df['cumulative_rank'] = df['cumulative_rank'] / df['rank']

# params to calculate metrics
output = {}
num_of_users = df.index.get_level_values('user_id').nunique()

# calc metrics
df[f'hit@{K}'] = df['rank'] <= K
output[f'Precision@{K}'] = (df[f'hit@{K}'] / K).sum() / num_of_users
output[f'Recall@{K}'] = (df[f'hit@{K}'] / df['users_watch_count']).sum() / num_of_users
output[f'MAP@{K}'] = (df["cumulative_rank"] / df["users_watch_count"]).sum() / num_of_users
print(f'Calculated metrics for top {K}')
output

Calculated metrics for top 10


{'Precision@10': 0.04552980132450331,
 'Recall@10': 0.4552980132450331,
 'MAP@10': 0.21939026695851052}

# Сабмит

In [12]:
submission_df = reranker_df[reranker_df['rank'] <= K].groupby(user_col)[item_col].apply(lambda x: ' '.join(x.astype(str))).reset_index()

In [13]:
submission_df.to_csv(os.path.join(data_path, 'sasrec_bert_reranker.csv'),index=False)